In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
import json, requests
from datetime import date
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import requests
from io import StringIO
import pandas as pd
import requests
from io import StringIO
import plotly.figure_factory as ff
import datetime


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# Any results you write to the current directory are saved as output.

![](https://www.diagnosticimaging.com/sites/default/files/covid-19.png)

# Table of Contents
*  [Timeline of Covid-19 in India](#1)
*  [India Covid-19 Numbers](#2)
*  [Recent Trends](#24)
*  [Covid-19 Cases Timeline](#3)
*  [Daily Confirmed Cases](#4)
*  [Analysis of States/UTs](#5)
*  [Top1,Top 5,Top 10 States/UTs Vs Rest Of India (RoI)](#6)
*  [Insights](#14)
*  [Recovered Cases and Deaths in States/UTs ](#7)
*  [Recovery Rate in States/UTs](#8)
*  [Death Rate in States/UTs](#9)
*  [Recovery Rate vs Tests/Million](#10)
*  [Insights](#13)
*  [Reccovery rate vs Death rate](#11)
*  [Insights](#12)
*  [Test Positivity Rate vs Tests/Million](#15)
*  [Insights](#16)
*  [Lockdown Analysis](#17)
*  [Absolute Comparison](#18)
*  [Percentage Comparison](#19)
*  [Insights](#20)
*  [Douling Days](#21)
*  [Insights](#22)
*  [Prediction Using DD](#23)

In [ ]:
# Common Methods
def State_Top_Analysis(number_top,input_df):
    result = input_df[:number_top]
    df1 = input_df.iloc[number_top:]
    result.loc['Rest of India'] = df1.sum()
    return(result)

def State_Top_Analysis2(number_top,input_df,top_number):
    result = input_df[0:0]
    df1 = input_df.iloc[:number_top]
    result.loc[top_number] = df1.sum()
    df2 = input_df.iloc[number_top:]
    result.loc['Rest of India'] = df2.sum()
    return(result)

def total_tests(startdate,enddate):
    #startdate = datetime.datetime.strptime(startdate , '%Y-%m-%d').date()
    #enddate = datetime.datetime.strptime(enddate , '%Y-%m-%d').date()
    startdate_tests = india_tested_numbers_data.loc[india_tested_numbers_data['Date']== startdate]['Total Samples Tested'].max()
    enddate_tests = india_tested_numbers_data.loc[india_tested_numbers_data['Date']== enddate]['Total Samples Tested'].max()
    totaltests = enddate_tests - startdate_tests
    return (totaltests)

def calculate_doublingtime(startdate,enddate):
    #startdate = datetime.datetime.strptime(startdate , '%Y-%m-%d').date()
    #enddate = datetime.datetime.strptime(enddate , '%Y-%m-%d').date()
    number_of_days = enddate - startdate
    cases_on_enddate =  case_time_series.loc[case_time_series['Casedate']== enddate]['Total Confirmed'].max()
    cases_on_startdate = case_time_series.loc[case_time_series['Casedate']== startdate]['Total Confirmed'].max()
    doublingrate = ((number_of_days.days*np.log(2))/(np.log(cases_on_enddate/cases_on_startdate)))
    doublingrate = np.round(doublingrate)
    return (doublingrate)

def doublingtime_predict(startdate,enddate,numberofdaystopredict):
    #startdate = datetime.datetime.strptime(startdate , '%Y-%m-%d').date()
    #enddate = datetime.datetime.strptime(enddate , '%Y-%m-%d').date()
    number_of_days = enddate - startdate
    cases_on_enddate =  case_time_series.loc[case_time_series['Casedate']== enddate]['Total Confirmed'].max()
    cases_on_startdate = case_time_series.loc[case_time_series['Casedate']== startdate]['Total Confirmed'].max()
    doublingrate = ((number_of_days.days*np.log(2))/(np.log(cases_on_enddate/cases_on_startdate)))
    doublingrate = np.round(doublingrate)
    predicteddoublingrate = cases_on_enddate * ((2**(1/doublingrate))** numberofdaystopredict)
    predicteddoublingrate = np.round(predicteddoublingrate,0)
    return (predicteddoublingrate)


#Global Variables 
width = 700
height = 550
colorscale = [[0, '#1f77b4'],[.5, '#f2e5ff'],[1, '#ffffff']]

In [ ]:
import requests
#Input DF1
url = "https://api.covid19india.org/csv/latest/case_time_series.csv"
req = requests.get(url)
data = StringIO(req.text)
case_time_series = pd.read_csv(data)
case_time_series = case_time_series.dropna()
case_time_series['year'] = 2020
case_time_series['Casedate']  =  case_time_series['Date'].astype(str)  + case_time_series['year'].astype(str) 
case_time_series['Casedate']  = pd.to_datetime(case_time_series['Casedate']).dt.date
case_time_series['Casedate']  = pd.to_datetime(case_time_series['Casedate'],format='%Y-%m-%d')
case_time_series['ConfirmedGrowthFactor'] = ( case_time_series['Daily Confirmed'] / case_time_series['Daily Confirmed'].shift(1))
case_time_series['RecoveredGrowthFactor'] = ( case_time_series['Daily Recovered'] / case_time_series['Daily Recovered'].shift(1))
case_time_series['DeathGrowthFactor'] = ( case_time_series['Daily Deceased'] / case_time_series['Daily Deceased'].shift(1))
#case_time_series = case_time_series.replace([np.inf, -np.inf], np.nan)
#case_time_series = case_time_series.fillna(0)

#Input DF2
url = "https://api.covid19india.org/csv/latest/state_wise.csv"
req = requests.get(url)
data = StringIO(req.text)
state_wise = pd.read_csv(data)


#Input DF3
url = "https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv"
req = requests.get(url)
data = StringIO(req.text)
statewise_tested_numbers_data = pd.read_csv(data)
statewise_tested_numbers_data['Date'] = statewise_tested_numbers_data['Updated On']
statewise_tested_numbers_data['Date'] = pd.to_datetime(statewise_tested_numbers_data['Date'], format='%d/%m/%Y')
statewise_tested_numbers_data.set_index("Date")
 
#Input DF4
url = "https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv"
req = requests.get(url)
data = StringIO(req.text)
india_tested_numbers_data = pd.read_csv(data)
india_tested_numbers_data['Date'] = india_tested_numbers_data['Update Time Stamp']
india_tested_numbers_data['Date'] = pd.to_datetime(india_tested_numbers_data['Update Time Stamp'],dayfirst=True).dt.date
india_tested_numbers_data['Date'] = pd.to_datetime(india_tested_numbers_data['Date'],format='%Y-%m-%d')
#india_tested_numbers_data

#DF4
total_covid_count = case_time_series[case_time_series['Casedate']==case_time_series['Casedate'].max()]
total_covid_count = total_covid_count.drop(['Casedate','Daily Confirmed','Daily Deceased','Daily Recovered','Date','year'],axis=1)
total_covid_count['National RecoveryRate'] = round(total_covid_count['Total Recovered'] / total_covid_count['Total Confirmed'],4) * 100
total_covid_count['National DeathRate'] = round(total_covid_count['Total Deceased'] /total_covid_count['Total Confirmed'], 4) * 100
mean = case_time_series['ConfirmedGrowthFactor'].mean()

# Timeline of Covid-19 in India  <a id="1"></a>

In [ ]:
start_date = case_time_series['Casedate'].min().date()
process_date = case_time_series['Casedate'].max().date()
number_of_days = process_date - start_date
print('Timeline of Covid-19 in India')
print('')
print('as of',case_time_series['Casedate'].max().strftime('%d/%m/%Y'))
print('')
print ('Date of First Covid-19 Case                    :',start_date.strftime('%d/%b/%Y'))
print('Number of days since the first Covid-19 Case   :',number_of_days.days ,'days')


# India Covid-19 Numbers  <a id="2"></a>

In [ ]:
date = case_time_series['Casedate'].max()
total_covid_count_bulletin1 = case_time_series[case_time_series['Casedate']== date ]
total_covid_count_bulletin1 = total_covid_count_bulletin1.drop(['Casedate','Daily Confirmed','Daily Deceased','Daily Recovered','Date','year','ConfirmedGrowthFactor','RecoveredGrowthFactor','DeathGrowthFactor'],axis=1)
total_covid_count_copy = total_covid_count_bulletin1.copy()
total_covid_count_bulletin2 = pd.DataFrame()
temp_df = case_time_series[case_time_series['Casedate']>='2020-03-14']
total_covid_count_bulletin2['National RecoveryRate'] = np.round(total_covid_count_bulletin1['Total Recovered'] / total_covid_count_bulletin1['Total Confirmed'],2) * 100
total_covid_count_bulletin2['National DeathRate'] = np.round(total_covid_count_bulletin1['Total Deceased'] /total_covid_count_bulletin1['Total Confirmed'], 2) * 100
total_covid_count_bulletin2['National Growth Factor (Mean)'] = np.round(temp_df['ConfirmedGrowthFactor'].mean(),2)
total_covid_count_bulletin2['Total Tests'] = np.round(india_tested_numbers_data['Total Samples Tested'].max(),2)
total_covid_count_bulletin2['Total Tests / Million'] = np.round(((total_covid_count_bulletin2['Total Tests']/1376000000)*1000000),0)
total_covid_count_bulletin2['TestPositivityRate'] = round(total_covid_count_bulletin1['Total Confirmed'] / total_covid_count_bulletin2['Total Tests'],2)*100

total_covid_count_bulletin1 = total_covid_count_bulletin1.to_string(index=False)
print('as of',date.strftime('%d/%b/%Y'))
print('')
print(total_covid_count_bulletin1)


In [ ]:
total_covid_count_bulletin3 = total_covid_count_bulletin2.drop(['Total Tests','Total Tests / Million','TestPositivityRate'],axis = 1)
total_covid_count_bulletin3 = total_covid_count_bulletin3.to_string(index=False)
print(total_covid_count_bulletin3)
#fig =  ff.create_table(total_covid_count_bulletin3, colorscale=colorscale)
#fig.show()

In [ ]:
total_covid_count_bulletin3 = total_covid_count_bulletin2.drop(['National Growth Factor (Mean)','National DeathRate','National RecoveryRate'],axis = 1)
total_covid_count_bulletin3 = total_covid_count_bulletin3.to_string(index=False)
print(total_covid_count_bulletin3)

# Recent Trends <a id="24"></a>

In [ ]:
data = {'':['ConfirmedCases','RecoveredCases','DeceasedCases','ActiveCases','RecoveryRate','DeathRate','GrowthFactor','TotalTests','TestPositivityRate']}   
virus_trends = pd.DataFrame(data)
last7daysdate = process_date -  pd.to_timedelta(6 , unit='d')
previous7daysdate = last7daysdate -  pd.to_timedelta(6 , unit='d')
temp_df = case_time_series[case_time_series['Casedate']>='2020-03-14']
overall =  case_time_series[case_time_series['Casedate'] <= process_date]
last7days = case_time_series[(case_time_series['Casedate'] >= last7daysdate) & (case_time_series['Casedate'] <= (process_date))]
previous7days = case_time_series[(case_time_series['Casedate'] >= previous7daysdate) & (case_time_series['Casedate'] <= (last7daysdate))]

virus_trends.ix [0,'Overall']         = overall ['Daily Confirmed'].sum()
virus_trends.ix [0,'Last 7 Days']     = last7days ['Daily Confirmed'].sum()
virus_trends.ix [0,'Previous 7 Days'] = previous7days ['Daily Confirmed'].sum()
virus_trends.ix [1,'Overall']         = overall ['Daily Recovered'].sum()
virus_trends.ix [1,'Last 7 Days']     = last7days ['Daily Recovered'].sum()
virus_trends.ix [1,'Previous 7 Days'] = previous7days ['Daily Recovered'].sum()
virus_trends.ix [2,'Overall']         = overall ['Daily Deceased'].sum()
virus_trends.ix [2,'Last 7 Days']     = last7days ['Daily Deceased'].sum()
virus_trends.ix [2,'Previous 7 Days'] = previous7days ['Daily Deceased'].sum()
virus_trends.ix [3,'Overall']         = [overall ['Daily Confirmed'].sum() - overall ['Daily Recovered'].sum() - overall ['Daily Deceased'].sum() ]
virus_trends.ix [3,'Last 7 Days']     = [last7days ['Daily Confirmed'].sum() - last7days ['Daily Recovered'].sum() - last7days ['Daily Deceased'].sum() ]
virus_trends.ix [3,'Previous 7 Days'] = [previous7days ['Daily Confirmed'].sum() - previous7days ['Daily Recovered'].sum() - previous7days ['Daily Deceased'].sum() ]
virus_trends.ix [4,'Overall']         = round(overall['Daily Recovered'].sum() / overall['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [4,'Last 7 Days']     = round(last7days['Daily Recovered'].sum() / last7days['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [4,'Previous 7 Days'] = round(previous7days['Daily Recovered'].sum() / previous7days['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [5,'Overall']         = round(overall['Daily Deceased'].sum() / overall['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [5,'Last 7 Days']     = round(last7days['Daily Deceased'].sum() / last7days['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [5,'Previous 7 Days'] = round(previous7days['Daily Deceased'].sum() / previous7days['Daily Confirmed'].sum(),4) * 100
virus_trends.ix [6,'Overall']         = round(temp_df['ConfirmedGrowthFactor'].mean(),4)
virus_trends.ix [6,'Last 7 Days']     = round(last7days['ConfirmedGrowthFactor'].mean(),4) 
virus_trends.ix [6,'Previous 7 Days'] = round(previous7days['ConfirmedGrowthFactor'].mean(),4) 
virus_trends.ix [7,'Overall']         = india_tested_numbers_data['Total Samples Tested'].max()
virus_trends.ix [7,'Last 7 Days']     = total_tests(last7daysdate,process_date)
virus_trends.ix [7,'Previous 7 Days'] = total_tests(previous7daysdate,last7daysdate)
virus_trends.ix [8,'Overall']         = round ((virus_trends.loc[0]['Overall']/ virus_trends.loc[7]['Overall']),4) * 100
virus_trends.ix [8,'Last 7 Days']     =  round ((virus_trends.loc[0]['Last 7 Days']/ virus_trends.loc[7]['Last 7 Days']),4) * 100
virus_trends.ix [8,'Previous 7 Days'] =  round ((virus_trends.loc[0]['Previous 7 Days']/ virus_trends.loc[7]['Previous 7 Days']),4) * 100
fig =  ff.create_table(virus_trends, colorscale=colorscale)
fig.show()


# Covid-19 Cases Timeline  <a id="3"></a>

In [ ]:
trace1 = go.Scatter(
                x=case_time_series['Casedate'],
                y=case_time_series['Total Confirmed'],
                name="Confirmed",
                mode='lines+markers',
                line_color='orange')
trace2 = go.Scatter(
                x=case_time_series['Casedate'],
                y=case_time_series['Total Deceased'],
                name="Deaths",
                mode='lines+markers',
                line_color='grey')

trace3 = go.Scatter(
                x=case_time_series['Casedate'],
                y=case_time_series['Total Recovered'],
                name="Recovered",
                mode='lines+markers',
                line_color='green')



layout = go.Layout(template="ggplot2", width=width, height=height, title_text = '<b>Spread of the Coronavirus In India Since jan 30 </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig = go.Figure(data = [trace1,trace2,trace3], layout = layout)
fig.show()




# Daily Confirmed Cases  <a id="4"></a>

In [ ]:
fig = px.bar(case_time_series, 
             x="Casedate", y="Daily Confirmed", 
             title='<b>Addition of new confirmed cases</b>', 
             orientation='v', 
             width=width, height=height)
fig.update_layout(template = 'ggplot2',font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig.show()

# Analysis of States/UTs  <a id="5"></a>

In [ ]:
state_cases = pd.pivot_table(state_wise, values=['Confirmed','Deaths','Recovered','Active'], index='State', aggfunc='max')
state_cases = state_cases [state_cases.index != 'Total']
state_cases = state_cases.sort_values(by='Active', ascending= False)
state_cases.style.background_gradient(cmap='Greys' ,subset=["Deaths"])\
.background_gradient(cmap='Blues' ,subset=["Active"])\
.background_gradient(cmap='YlOrRd' ,subset=["Confirmed"])\
.background_gradient(cmap='Greens' ,subset=["Recovered"])


# Top1,Top 5,Top 10 States Vs Rest Of India (RoI)<a id="6"></a>

In [ ]:
State_Top1 = State_Top_Analysis2(1,state_cases,'Maharastra')
State_Top5 = State_Top_Analysis2(5,state_cases,'Top 5 States')
State_Top10 = State_Top_Analysis2(10,state_cases,'Top 10 States')

fig = make_subplots (rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"},{"type": "pie"}]],
                     subplot_titles=['Maharastra vs ROI', 'Top 5  vs ROI',
                                     'Top 10  vs ROI'])
trace1 = go.Pie(
                labels=State_Top1.index,
                values=State_Top1.Active,
                hoverinfo='label+percent+value', 
                textfont_size=12,
                name = 'graph1',
                marker=dict(line=dict(color='#000000', width=2)))        
fig.append_trace(trace1, 1, 1)

trace2 = go.Pie(
                labels=State_Top5.index,
                values=State_Top5.Active,
                hoverinfo='label+percent+value', 
                textfont_size=12,
                name ='graph 2',
                marker=dict(line=dict(color='#000000', width=2)))                         
fig.append_trace(trace2, 1, 2)

trace3 = go.Pie(
                labels=State_Top10.index,
                values=State_Top10.Active,
                hoverinfo='label+percent+value', 
                textfont_size=12,
                name ='graph 2',
                marker=dict(line=dict(color='#000000', width=2)))                         
fig.append_trace(trace3, 1, 3)



layout = go.Layout(width=width, height=height,title = '<b>Confirmed Cases : Top 10 States VS Rest of India </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))

fig = go.Figure(fig)
fig.show()


  # **Insights :** <a id="14"></a>
1. <n> *Maharastra has 25 out of 100 Total confirmed cases than Rest of India*
2. <n> Every 38 out of 100  confirmed casses is from Top 5 states w.r.t Table above 
3. <n> *Every 78 out of 100 confirmed casses is from Top 10 states w.r.t Table above*
   
  
 

# Recovered Cases and Deaths in States/UTs <a id="7"></a>

In [ ]:
state_cases_deaths = state_cases.sort_values(by='Deaths', ascending= False)
state_cases_cured = state_cases.sort_values(by='Recovered', ascending= False)
State_Top10_breakup_cured  = State_Top_Analysis(10,state_cases_cured)
State_Top10_breakup_deaths  = State_Top_Analysis(10,state_cases_deaths)

fig = make_subplots (rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]],
                     subplot_titles=['Recovered cases in States/UTs', 'Deaths in States/UTs'])
trace1 = go.Pie(
                labels=State_Top10_breakup_cured.index,
                values=State_Top10_breakup_cured.Recovered,
                hoverinfo='label+percent+value', 
                textfont_size=12,
                name = 'graph1',
                marker=dict(line=dict(color='#000000', width=2)))
                            
fig.append_trace(trace1, 1, 1)

colors = ['paired']
trace2 = go.Pie(
                labels=State_Top10_breakup_deaths.index,
                values=State_Top10_breakup_deaths.Deaths,
                hoverinfo='label+percent+value', 
                textfont_size=12,
                name ='graph 2',
                marker=dict(line=dict(color='#000000', width=2)))
                            
fig.append_trace(trace2, 1, 2)

layout = go.Layout(width=width, height=height,title_text = '<b>Confirmed Cases : Top 10 States VS Rest of India </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))

fig = go.Figure(fig)
fig.show()


# Recovery Rate and Death Rate
* <n> Recovery rate is the percentage of patients who recovered on total confirmed cases
* <n> Death rate is the percentage of patients who died on total confirmed cases
* <n> Only states and UTs who are in top 20 have individual entries.Rest all are combined together

# Recovery Rate in States/UTs <a id="8"></a>

In [ ]:
State_Top20_breakup = State_Top_Analysis(21,state_cases)
State_Top20_breakup['State/UT'] = State_Top20_breakup.index
State_Top20_breakup['RecoveryRate'] = np.round((State_Top20_breakup['Recovered'] / State_Top20_breakup['Confirmed']) *100 , decimals =1)
State_Top20_breakup['DeathRate'] = np.round((State_Top20_breakup["Deaths"]/State_Top20_breakup["Confirmed"]) *100,decimals=1)

State_Top20_breakup_recoveryrate = State_Top20_breakup.sort_values('RecoveryRate', ascending= True)


fig = px.bar(State_Top20_breakup_recoveryrate,
             x="RecoveryRate", y="State/UT", 
             title='Recovery Rate in States/UTs', 
             text='RecoveryRate',
             orientation='h', 
             width=width, height=900)
fig.update_traces(marker_color='Green', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()



# Death Rate in States/UTs <a id="9"></a>

In [ ]:
State_Top20_breakup_deathrate = State_Top20_breakup.sort_values('DeathRate', ascending= True)
fig = px.bar(State_Top20_breakup_deathrate,
             x="DeathRate", y="State/UT", 
             title='Death Rate in States / UTs', 
             text='DeathRate', 
             orientation='h', 
             width=width, height=900)
fig.update_traces(marker_color='grey', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()

# Recovery Rate vs Tests/Million (Confirmed Cases) <a id="10"></a>

In [ ]:
#DF for State testing
State_testing = pd.pivot_table(statewise_tested_numbers_data, values=['Test positivity rate','Total Tested' ,'Tests per thousand','Tests per million'], index='State', aggfunc='max')
State_testing = State_testing.sort_values('Total Tested', ascending= False)
State_testing = State_testing.combine_first(state_cases)
State_testing ['Recoveryrate'] = np.round((State_testing['Recovered'] / State_testing['Confirmed']) *100 , decimals =1)
State_testing['DeathRate'] = np.round((State_testing["Deaths"]/State_testing["Confirmed"]) *100,decimals=1)
State_testing['TestpositivityRate'] = np.round((State_testing['Confirmed'] / State_testing['Total Tested'])*100,decimals=1)
State_testing = State_testing.dropna(axis=0)
State_testing = State_testing.astype(float)

#Split into top10 and 10-20
State_testing = State_testing.sort_values('Confirmed', ascending= False)
State_confirmed_1to10 = State_testing[0:10]
State_confirmed_10to20 = State_testing[11:21]

#For National Recovery rate line
nationalrr = total_covid_count_bulletin2['National RecoveryRate'].values
statetop20tests = State_testing [0:21]
data = {'x':[nationalrr,nationalrr], 'y':[0,max(statetop20tests['Tests per million'])]} 
df_recoveryrate = pd.DataFrame(data) 
df_recoveryrate = df_recoveryrate.astype(float)

fig = go.Figure()
trace1 =go.Scatter(
                    x = State_confirmed_1to10['Recoveryrate'],
                    y = State_confirmed_1to10['Tests per million'],
                    mode = "markers",  
                    line_color='Red',
                    name = 'First 10 Confirmed States',
                    text= State_confirmed_1to10.index)
trace2 =go.Scatter(
                    x = State_confirmed_10to20['Recoveryrate'],
                    y = State_confirmed_10to20['Tests per million'],
                    mode = "markers",  
                    line_color='orange',
                    name = '[10 - 20] Confirmed States',
                    text= State_confirmed_10to20.index)

trace3 =go.Scatter(
                    x = df_recoveryrate['x'],
                    y = df_recoveryrate['y'],
                    mode = "lines",  
                    name="National Recoveryrate",
                    line_color='green',
                    text = total_covid_count_bulletin2['National RecoveryRate'])

layout = dict(
              
              xaxis= dict(title= 'Recovery Rate',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Tests per million',ticklen= 5,zeroline= False)
             )

fig = go.Figure(data = [trace1,trace2,trace3], layout = layout)



fig.show()



   <n> *In the above scatter plot*
1. <n> *Green line represents the National Recovery Rate*
2. <n> *Red Dot represents the First 10 States/UT in terms of Confirmed cases*
3. <n> *Yellow Dot represents the States/UT in rank from 10 to 20*
   
**<n> *Ideal Scenario***

* <n> *Anything to the right of the National Recovery Rate is Good. More away from the green line on right more the recovery rate *
* <n> *Tests/Million vary on basis of Covid spread , Population Density , Testing centres etc *
* <n> *More higher in the graph the better for the states*
* <n> *When States/UT discharge more patients the green line will move towards the right *
* <n> *When the active cases rise the green line will move towards the left* 
    

 # **Insights :** <a id="13"></a>
1. <n> *5 States/UT in First 10 list are behind the national recovery rate*
2. <n> *6 States/UT in Rank [10 – 20] are behind the national recovery rate*
3. <n> *7 States/UT have recovery rate above 80 Percentage*


# Recovery Rate vs Death Rate (Active Cases)<a id="11"></a>

In [ ]:
#Rearrange DF to active cases
State_testing = State_testing.sort_values('Active', ascending= False)
State_active_1to10 = State_testing[0:10]
State_active_10to20 = State_testing[11:21]
nationalrr = total_covid_count_bulletin2['National RecoveryRate'].values
nationaldr = total_covid_count_bulletin2['National DeathRate'].values
staterr = max(State_testing['DeathRate'])
data = {'x':[nationalrr,nationalrr], 'y':[0,staterr]} 
df_State_rr = pd.DataFrame(data) 
df_State_rr = df_State_rr.astype(float)

#natioanl Death Rate line
data = {'x':[50,100], 'y':[nationaldr,nationaldr]} 
df_State_dr = pd.DataFrame(data) 
df_State_dr = df_State_dr.astype(float)

trace1 =go.Scatter(
                    x = State_active_1to10['Recoveryrate'],
                    y = State_active_1to10['DeathRate'],
                    mode = "markers",  
                    name = 'Top 10 Active States',
                    line_color='Red',
                    text= State_active_1to10.index)
trace2 =go.Scatter(
                    x = State_active_10to20['Recoveryrate'],
                    y = State_active_10to20['DeathRate'],
                    mode = "markers",  
                    name = '[10-20] Active States',
                    line_color='Orange',
                    text= State_active_10to20.index)

trace3 =go.Scatter(
                    x = df_State_rr['x'],
                    y = df_State_rr['y'],
                    mode = "lines",  
                    name="National Recoveryrate",
                    line_color='green',
                    text = total_covid_count_bulletin2['National RecoveryRate'])
trace4 =go.Scatter(
                    x = df_State_dr['x'],
                    y = df_State_dr['y'],
                    mode = "lines",  
                    name="National Death Rate",
                    line_color='grey',
                    text = total_covid_count_bulletin2['National DeathRate'])

data = [trace1,trace2,trace3,trace4]
layout = dict(
              xaxis= dict(title= 'Recovery Rate',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Death rate',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)



<n> *In the above scatter plot*
1. <n> *Green line represents the National Recovery Rate*
2. <n> *Grey line represents the National Death Rate*
3. <n> *Red Dot represents the First 10 States/UT in terms of Confirmed cases*
4. <n> *Yellow Dot represents the States/UT in rank from 10 to 20*
5. <n> *The Graph has 4 Quadrants [Q1,Q2,Q3,Q4] *
6. <n> *Q1 – Top Right – High Recovery Rate and High Death Rate *
7. <n> *Q2 – Top Left – Low Recovery Rate and High Death Rate *
8. <n> *Q3 – Bottom left – Low Recovery Rate and Low Death Rate *
9. <n> * Q4 – Bottom Right  - High Recovery Rate and Low Death Rate* 
    
**<n> *Ideal Scenario***
* <n> *States/UT Needs to target to be in Q4*
* <n> *States/UT should not be in Q2*

<n>
 # **Insights :**  <a id="12"></a>
1. <n> *5 of the top 20 States/UT are in Q4 (High Recovery Rate and Low Death Rate).This reflects India’s lower mortality rate in the world*
2. <n> * 3 States/UT are in Q2 (Low Recovery Rate and High Death Rate)*
3. <n> *3 States/UT are in Q1 (High Recovery Rate and High Death Rate)*
4. <n> * 11 States/UT are in Q3 (Low Recovery Rate and Low Death Rate)*


# TestPositivity vs Tests/Million (Confirmed Cases)<a id="15"></a>

In [ ]:
#National Test Positivity line
nationaltestpositivity = total_covid_count_bulletin2['TestPositivityRate'].values
data = {'x':[nationaltestpositivity,nationaltestpositivity], 'y':[0,max(statetop20tests['Tests per million'])]} 
df_testpositive = pd.DataFrame(data) 
df_testpositive = df_testpositive.astype(float)

#National Test/Milllion Line
nationaltestmillion = total_covid_count_bulletin2['Total Tests / Million'].values
data = {'x':[0,max(statetop20tests['TestpositivityRate'])], 'y':[nationaltestmillion,nationaltestmillion]} 
df_testmillion = pd.DataFrame(data) 
df_testmillion = df_testmillion.astype(float)


fig = go.Figure()
trace1 =go.Scatter(
                    x = State_confirmed_1to10['TestpositivityRate'],
                    y = State_confirmed_1to10['Tests per million'],
                    mode = "markers",  
                    line_color='Red',
                    name = 'First 10 Confirmed States',
                    text= State_confirmed_1to10.index)
trace2 =go.Scatter(
                    x = State_confirmed_10to20['TestpositivityRate'],
                    y = State_confirmed_10to20['Tests per million'],
                    mode = "markers",  
                    line_color='orange',
                    name = '[10 - 20] Confirmed States',
                    text= State_confirmed_10to20.index)

trace3 =go.Scatter(
                    x = df_testpositive['x'],
                    y = df_testpositive['y'],
                    mode = "lines",  
                    name="National Test Positivity Rate",
                    line_color='green',
                    text = total_covid_count_bulletin2['TestPositivityRate'])
trace4 =go.Scatter(
                    x = df_testmillion['x'],
                    y = df_testmillion['y'],
                    mode = "lines",  
                    name="National Tests/Million",
                    line_color='grey',
                    text = total_covid_count_bulletin2['Total Tests / Million'])

layout = dict(
              
              xaxis= dict(title= 'Test Positivity Rate',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Tests per million',ticklen= 5,zeroline= False)
             )



fig = go.Figure(data = [trace1,trace2,trace3,trace4], layout = layout)



fig.show()


<n> *In the above scatter plot*
1. <n> *Green line represents the National Test Positivity Rate*
2. <n> *Grey line represents the National Tests/Million *
3. <n> *Red Dot represents the First 10 States/UT in terms of Confirmed cases*
4. <n> *Yellow Dot represents the States/UT in rank from 10 to 20 in terms of Confirmed cases*
5. <n> *The Graph has 4 Quadrants [Q1,Q2,Q3,Q4] *
6. <n> *Q1 – Top Right – High Test Positivity Rate and High Tests/Million *
7. <n> *Q2 – Top Left – Low Test Positivity Rate and High Tests/Million *
8. <n> *Q3 – Bottom left – Low Test Positivity Rate and Low Tests/Million *
9. <n> * Q4 – Bottom Right  - High Test Positivity Rate and Low Tests/Million* 
    
**<n> *Ideal Scenario***
* <n> *States/UT Needs to target to be in Q2*
* <n> *States/UT should not be in Q4*

<n>
 # **Insights :**  <a id="16"></a>
1. <n> * 1 States/UT are in Q2 (Low Test Positivity Rate and High Tests/Million)*
2. <n> * 12 States/UT are in Q3 (Low Test Positivity Rate and Low Tests/Million)*
3. <n> * 5 States/UT are in Q4  (High Test Positivity Rate and Low Tests/Million)*
4. <n> * 2 States/UT are in Q1 (High Test Positivity Rate and High Tests/Million)*


# Lockdown Analysis<a id="17"></a>




*On 24 March 2020, the Government of India under Prime Minister Narendra Modi ordered a nationwide lockdown for 21 days, limiting movement of the entire 1.3 billion population of India as a preventive measure against the COVID-19 pandemic in India.It was ordered after a 14-hour voluntary public curfew on 22 March, followed by enforcement of a series of regulations in the country's COVID-19 affected regions. The lockdown was placed when the number of confirmed positive coronavirus cases in India was approximately 500*

In [ ]:
start_date = '2020-03-25'
start_date = pd.to_datetime(start_date)
process_date = case_time_series['Casedate'].max().date()
number_of_days = process_date - start_date.date()
print('Timeline of Lockdown in India')
print('')
print('as of',case_time_series['Casedate'].max().strftime('%d/%m/%Y'))
print('')
print ('Date of First lockdown                             :',start_date.strftime('%d/%b/%Y'))
print('Number of days in lockdown (Inc Partial lockdowns) :',number_of_days.days ,'days')
print('Current Lockdown Version                           : Unlock 4.0')


**<n>*Lockdown Phases***
<n>
<n>Phase 1 - Lockdown 1.0 - 25 March 2020 – 14 April 2020 (21 days)
<n>Phase 2 - Lockdown 2.0 - 15 April 2020 – 3  May 2020 (19 days)
<n>Phase 3 - Lockdown 3.0 - 4  May   2020 – 17 May 2020 (14 days)
<n>Phase 4 - Lockdown 4.0 - 18 May   2020 – 31 May 2020 (14 days)
<n>Phase 5 - Unlock 1.0 - 1  June  2020 – 30 June 2020 (30 days)
<n>Phase 6 - Unlock 2.0 - 1  July  2020 – 31 July 2020 (31 days)
<n>Phase 7 - Unlock 3.0 - 1  Aug  2020 – 31 Aug 2020 (31 days)
<n>Phase 8 - Unlock 4.0 - 1  Sep  2020 – 30 Sep 2020 (30 days)    
    
<n>India is currently in Unlock 4.0 till Sep 30,2020

In [ ]:
data = {'Lockdown_Version':['Lockdown 1.0','Lockdown 2.0','Lockdown 3.0','Lockdown 4.0' ,'Unlock 1.0','Unlock 2.0','Unlock 3.0','Unlock 4.0'],'No of Days':[21,19,14,14,30,31,31,30]}   
lockdown_df = pd.DataFrame(data)
#cases_lockdown
lockdown1_cases = case_time_series[(case_time_series['Casedate'] >= '2020-03-25') & (case_time_series['Casedate'] <= '2020-04-14')]
lockdown2_cases = case_time_series[(case_time_series['Casedate'] >= '2020-04-15') & (case_time_series['Casedate'] <= '2020-05-03')]
lockdown3_cases = case_time_series[(case_time_series['Casedate'] >= '2020-05-04') & (case_time_series['Casedate'] <= '2020-05-17')]
lockdown4_cases = case_time_series[(case_time_series['Casedate'] >= '2020-05-18') & (case_time_series['Casedate'] <= '2020-05-31')]
lockdown5_cases = case_time_series[(case_time_series['Casedate'] >= '2020-06-01') & (case_time_series['Casedate'] <= '2020-06-30')]
lockdown6_cases = case_time_series[(case_time_series['Casedate'] >= '2020-07-01') & (case_time_series['Casedate'] <= '2020-07-31')]
lockdown7_cases = case_time_series[(case_time_series['Casedate'] >= '2020-08-01') & (case_time_series['Casedate'] <= '2020-08-31')]
lockdown8_cases = case_time_series[(case_time_series['Casedate'] >= '2020-09-01') & (case_time_series['Casedate'] <= '2020-09-30')]
lockdown_df.ix [0,'ConfirmedCases'] = lockdown1_cases['Daily Confirmed'].sum()
lockdown_df.ix [1,'ConfirmedCases'] = lockdown2_cases['Daily Confirmed'].sum()
lockdown_df.ix [2,'ConfirmedCases'] = lockdown3_cases['Daily Confirmed'].sum()
lockdown_df.ix [3,'ConfirmedCases'] = lockdown4_cases['Daily Confirmed'].sum()
lockdown_df.ix [4,'ConfirmedCases'] = lockdown5_cases['Daily Confirmed'].sum()
lockdown_df.ix [5,'ConfirmedCases'] = lockdown6_cases['Daily Confirmed'].sum()
lockdown_df.ix [6,'ConfirmedCases'] = lockdown7_cases['Daily Confirmed'].sum()
lockdown_df.ix [7,'ConfirmedCases'] = lockdown8_cases['Daily Confirmed'].sum()
lockdown_df['Confirmed/Day'] = round (lockdown_df['ConfirmedCases'] / lockdown_df['No of Days'],0)
lockdown_df.ix [0,'Recovered'] = lockdown1_cases['Daily Recovered'].sum()
lockdown_df.ix [1,'Recovered'] = lockdown2_cases['Daily Recovered'].sum()
lockdown_df.ix [2,'Recovered'] = lockdown3_cases['Daily Recovered'].sum()
lockdown_df.ix [3,'Recovered'] = lockdown4_cases['Daily Recovered'].sum()
lockdown_df.ix [4,'Recovered'] = lockdown5_cases['Daily Recovered'].sum()
lockdown_df.ix [5,'Recovered'] = lockdown6_cases['Daily Recovered'].sum()
lockdown_df.ix [6,'Recovered'] = lockdown7_cases['Daily Recovered'].sum()
lockdown_df.ix [7,'Recovered'] = lockdown8_cases['Daily Recovered'].sum()
lockdown_df['Recovered/Day'] = round (lockdown_df['Recovered'] / lockdown_df['No of Days'],0)
lockdown_df.ix [0,'Deceased'] = lockdown1_cases['Daily Deceased'].sum()
lockdown_df.ix [1,'Deceased'] = lockdown2_cases['Daily Deceased'].sum()
lockdown_df.ix [2,'Deceased'] = lockdown3_cases['Daily Deceased'].sum()
lockdown_df.ix [3,'Deceased'] = lockdown4_cases['Daily Deceased'].sum()
lockdown_df.ix [4,'Deceased'] = lockdown5_cases['Daily Deceased'].sum()
lockdown_df.ix [5,'Deceased'] = lockdown6_cases['Daily Deceased'].sum()
lockdown_df.ix [6,'Deceased'] = lockdown7_cases['Daily Deceased'].sum()
lockdown_df.ix [7,'Deceased'] = lockdown8_cases['Daily Deceased'].sum()
lockdown_df['Deceased/Day'] = round (lockdown_df['Deceased'] / lockdown_df['No of Days'],0)
lockdown_df.ix [0,'TotalTests'] = total_tests('2020-03-25','2020-04-14')
lockdown_df.ix [1,'TotalTests'] = total_tests('2020-04-15','2020-05-03')
lockdown_df.ix [2,'TotalTests'] = total_tests('2020-05-04','2020-05-17')
lockdown_df.ix [3,'TotalTests'] = total_tests('2020-05-17','2020-05-31')
lockdown_df.ix [4,'TotalTests'] = total_tests('2020-06-01','2020-06-30')
lockdown_df.ix [5,'TotalTests'] = total_tests('2020-07-01','2020-07-31')
lockdown_df.ix [6,'TotalTests'] = total_tests('2020-08-01','2020-08-31')
lockdown_df.ix [7,'TotalTests'] = total_tests('2020-09-01','2020-09-02')
lockdown_df['Tests/Day'] = round (lockdown_df['TotalTests'] / lockdown_df['No of Days'],0)
lockdown_df['Tests/Million'] = round(((lockdown_df['TotalTests']/1376000000)*1000000),0)
lockdown_df.ix [0,'GrowthFactor'] = round (lockdown1_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [1,'GrowthFactor'] = round (lockdown2_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [2,'GrowthFactor'] = round (lockdown3_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [3,'GrowthFactor'] = round (lockdown4_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [4,'GrowthFactor'] = round (lockdown5_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [5,'GrowthFactor'] = round (lockdown6_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [6,'GrowthFactor'] = round (lockdown7_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df.ix [7,'GrowthFactor'] = round (lockdown8_cases['ConfirmedGrowthFactor'].mean(),2)
lockdown_df['RecoveryRate'] = round(100 * (lockdown_df['Recovered'] / lockdown_df['ConfirmedCases']),2)
lockdown_df['DeathRate'] =  round (100 * (lockdown_df['Deceased'] / lockdown_df['ConfirmedCases']),2)
lockdown_df['TestPositivityRate'] =  round (100 * (lockdown_df['ConfirmedCases'] / lockdown_df['TotalTests']),2)


# Absolute Comparison <a id="18"></a>

<n> The table below lists out the Confirmedcases, Recovered, Deceased and Totaltests across all lockdowns
<n>
<n> Description :
<n>
1.     ConfimedCases - Number of confirmed cases in the particular lockdown
2.     Recovered     - Number of people recovered in the particular lockdown
3.     Deceased      - Number of people dead in the particular lockdown
4.     TotalTests    - Number of Tests conducted in the particular lockdown
    
All the values are calculated by finding the differnce between start date and end date of lockdown

In [ ]:
lockdown_cases = lockdown_df[['Lockdown_Version','No of Days','ConfirmedCases','Recovered','Deceased','TotalTests']].copy()
fig =  ff.create_table(lockdown_cases, colorscale=colorscale)
fig.show()

*Since the number of days in lockdowns are not consistant hence lets calculate on per day basis*


In [ ]:
lockdown_cases2 = lockdown_df[['Lockdown_Version','No of Days','Confirmed/Day','Recovered/Day','Deceased/Day','Tests/Day']].copy()
fig =  ff.create_table(lockdown_cases2, colorscale=colorscale)
fig.show()

# Percentage Comparison <a id="19"></a>

<n> The table below lists out the Tests/Million,Growthfactor,RecoveryRate,DeathRate and TestPositivityRate across all lockdowns
<n>
<n> Description :
<n>
1.     Tests/Million        - Number of tests performed on a million population
2.     Growthfactor         - Average growth of confirmeed cases on a day by day basis
3.     RecoveryRate         - Number of people recovered on Number of people with covid
4.     DeathRate            - Number of people dead on Number of people with covid
5.     TestPositivityRate   - Number of tests positive on total number of tests
    

In [ ]:
lockdown_cases3 = lockdown_df[['Lockdown_Version','Tests/Million','GrowthFactor','RecoveryRate','DeathRate','TestPositivityRate']].copy()
fig =  ff.create_table(lockdown_cases3, colorscale=colorscale)
fig.show()

 # **Insights :**  <a id="20"></a>
 
**Good** 
1. <n> *Testing - Increase in Number of tests and thereby increasing Tests/Million*
2. <n> *Steady Increase in Recovery rate*
3. <n> *Decrease in Death rate*
4. <n> *Growth factor in declining trend*

**Bad** 
1. <n> *Increase in number of confirmed cases which can overwhelme healthcare infrastucture*
2. <n> *Deaths*


 # Doubling Days<a id="21"></a>
 <n>Doubling time is time it takes for a desease to double in current size/value

In [ ]:
startdate = datetime.datetime.strptime('2020-03-01' , '%Y-%m-%d').date()
#enddate = case_time_series['Casedate'].max().date()
enddate = datetime.datetime.strptime('2020-09-02' , '%Y-%m-%d').date()
weekfreq = pd.date_range(start=startdate ,end=enddate, freq='W')
week_df = pd.DataFrame(columns=['Startdate'])
week_df['Startdate'] = weekfreq
week_df['Enddate'] = week_df.shift(-1)
week_df['Enddate']  =week_df['Enddate'] - datetime.timedelta(days=1)
for i in range (len(weekfreq)):
    week_df.ix[i,'DoublingDays'] = calculate_doublingtime(week_df.ix[i,'Startdate'],week_df.ix[i,'Enddate'])    
    week_df.ix[i,'week'] = 'Week' + str(i)
    week_df.ix[i,'PredictedCount'] = doublingtime_predict(week_df.ix[i,'Startdate'],week_df.ix[i,'Enddate'],7)
    week_df.ix[i,'ActualCount'] = case_time_series.loc[case_time_series['Casedate'] == week_df.ix[i,'Enddate']]['Total Confirmed'].max()
week_df.ix[0,'week'] = '01/Mar/2020'   
week_df['PredictedCount'] = week_df['PredictedCount'].shift(1)
weekly_df_doubling = week_df[['Startdate','Enddate','DoublingDays']].copy()
weekly_df_doubling['Startdate'] =pd.to_datetime(weekly_df_doubling['Startdate']).dt.date
weekly_df_doubling['Enddate'] =pd.to_datetime(weekly_df_doubling['Enddate']).dt.date
weekly_df_doubling = weekly_df_doubling[weekly_df_doubling['Enddate'] <= case_time_series['Casedate'].max().date()]
#fig =  ff.create_table(weekly_df_doubling, colorscale=colorscale)
#fig.show()

In [ ]:

trace1 = go.Scatter(
                x=week_df['week'],
                y=weekly_df_doubling['DoublingDays'],
                name="Confirmed",
                mode='lines+markers',
                line_color='orange')

layout = go.Layout(template="ggplot2", width=width, height=height, title_text = '<b>Douling Days since March 01</b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig = go.Figure(data = [trace1], layout = layout)
fig.show()


 # **Insights :** <a id="22"></a>
Doubling rate is continuously increasing since March 1,2020
   


 # Prediction of Confirmed Cases<a id="23"></a>
 <n>By Reversing the doubling days equation we can predict the total confirmed cases on the particular day

In [ ]:
weekly_df_doubling_predict = week_df[['Enddate','PredictedCount','ActualCount']].copy()
weekly_df_doubling_predict['Enddate'] =pd.to_datetime(weekly_df_doubling_predict['Enddate']).dt.date
weekly_df_doubling_predict = weekly_df_doubling_predict[weekly_df_doubling_predict['Enddate'] <= enddate]
weekly_df_doubling_predict = weekly_df_doubling_predict[10:]
fig =  ff.create_table(weekly_df_doubling_predict, colorscale=colorscale)
fig.show()

<n>If the Actual confirmed cases is higher than the predicted value we can assume that Doubling days has reduced
 <n>If the Actual confirmed cases is lower than the predicted value we can assume that Doubling days has increased
 <n>More the doubling days more the better

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar(name='Predicted', x=weekly_df_doubling_predict['Enddate'], y=weekly_df_doubling_predict['PredictedCount']),
    go.Bar(name='Actual', x=weekly_df_doubling_predict['Enddate'], y=weekly_df_doubling_predict['ActualCount'])
])

fig.update_layout(barmode='group')
fig.show()

* **Upcoming**
<n>ML Prediction Model

**Version Details **
<n>1.0 - Initial analysis and Visualizations - 22/Apr/2020
<n>2.0 - Changed Data source , Additional Visualizations - 16/May/2020
<n>3.0 - National Test Details , Growth Factor  - 26/May/2020   
<n>4.0 - TestPositivityRate vs Tests/Million  - 28/May/2020      
<n>5.0 - Lockdown Analysis - 01/Jun/2020    
<n>6.0 - Doubling Days and Prediction of Confimed cases using DD - 14/Jun/2020 
<n>7.0 - Recent Trends (2 Week anlysis)  - 17/Aug/2020
<br>
    <br>
    Your valuable feedback will be appreciated


![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Covid-19-curves-graphic2-stopthespread-v3.gif/800px-Covid-19-curves-graphic2-stopthespread-v3.gif)


